# **Tumor Detector 2.0 - Architectures**
---

### **Imports**

In [1]:
# layers
from tensorflow.keras.layers import Conv2D,\
                                    UpSampling2D,\
                                    MaxPooling2D,\
                                    Activation,\
                                    BatchNormalization,\
                                    Concatenate,\
                                    Input,\
                                    Multiply,\
                                    Add

# data handler
import tensorflow as tf

# system
import os

In [2]:
# variables
OUTPUT_PATH = '../models'

### **Blocks**

##### **Attention Gate**

In [4]:
def attention_gate(g, x):

  # saving output shape
  init_filters = x.shape[-1]

  # applying sum
  concat = Add()([
      Conv2D(filters=init_filters, kernel_size=1, strides=1)(g),
      Conv2D(filters=init_filters, kernel_size=1, strides=2, padding='same')(x)
  ])

  # applying ReLU
  relu = Activation(activation='relu')(concat)

  # multiplying weights
  mult_weights = Conv2D(filters=1, kernel_size=1, strides=1)(relu)

  # normalizing weights
  sigmoid = Activation(activation='sigmoid')(mult_weights)

  # upsampling matrix
  up_sample = UpSampling2D(size=2)(sigmoid)

  # multiply by x
  output = Multiply()([up_sample, x])

  return output

##### **Up/Down Sampling**

In [5]:
def down_sampling_block(x, filter):

  x = Conv2D(filters=filter, kernel_size=3, padding='same')(x)
  x = BatchNormalization(axis=-1)(x)
  x = Activation(activation='relu')(x)

  x = Conv2D(filters=filter, kernel_size=3, padding='same')(x)
  x = BatchNormalization(axis=-1)(x)
  x = Activation(activation='relu')(x)

  down = MaxPooling2D(pool_size=2, strides=2)(x)

  return x, down

def up_sampling_block(x, filter):

  x = Conv2D(filters=filter, kernel_size=3, padding='same')(x)
  x = BatchNormalization(axis=-1)(x)
  x = Activation(activation='relu')(x)

  x = Conv2D(filters=filter, kernel_size=3, padding='same')(x)
  x = BatchNormalization(axis=-1)(x)
  x = Activation(activation='relu')(x)

  up = UpSampling2D(size=2)(x)

  return x, up

### **tumorNet_v1**

In [ ]:
INPUT = Input(shape=[256, 256, 1], name='input-layer')

# down-sampling
x_1, down_1 = down_sampling_block(INPUT, 64)                                    # 256x256x1   ->  ( 256x256x64  | 128x128x64)
x_2, down_2 = down_sampling_block(down_1, 128)                                  # 128x128x64  ->  ( 128x128x128 | 64x64x128 )
x_3, down_3 = down_sampling_block(down_2, 256)                                  # 64x64x128   ->  ( 64x64x256   | 32x32x256 )
x_4, down_4 = down_sampling_block(down_3, 512)                                  # 32x32x256   ->  ( 32x32x512   | 16x16x512 )

# up-sampling
g_4, up_1 = up_sampling_block(down_4, 1024)                                     # 16x16x512   ->  ( 16x16x1024  | 32x32x1024)
attention_1 = attention_gate(g_4, x_4)                                          # (   16x16x1024|   32x32x512 ) ->  32x32x512
concat = Concatenate(axis=-1)([attention_1, up_1])                              # (   32x32x512 |   32x32x1024) ->  32x32x1536

g_3, up_2 = up_sampling_block(concat, 512)                                      # 32x32x1536  ->  ( 32x32x512   | 64x64x512 )
attention_2 = attention_gate(g_3, x_3)                                          # (   32x32x512 |   64x64x256 ) ->  64x64x256
concat = Concatenate(axis=-1)([attention_2, up_2])                              # (   64x64x256 |   64x64x512 ) ->  64x64x768

g_2, up_3 = up_sampling_block(concat, 256)                                      # 64x64x768   ->  ( 64x64x256   | 128x128x256)
attention_3 = attention_gate(g_2, x_2)                                          # (   64x64x256 | 128x128x128 ) ->  128x128x128
concat = Concatenate(axis=-1)([attention_3, up_3])                              # ( 128x128x128 | 128x128x256 ) ->  128x128x384

g_1, up_4 = up_sampling_block(concat, 128)                                      # 128x128x384 ->  ( 128x128x128 | 256x256x128)
attention_4 = attention_gate(g_1, x_1)                                          # ( 128x128x128 | 256x256x64  ) ->  256x256x64
concat = Concatenate(axis=-1)([attention_4, up_4])                              # ( 256x256x64  | 256x256x128 ) ->  256x256x192

# final block
x = Conv2D(filters=64, kernel_size=3, padding='same')(concat)                   # 256x256x192 ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)                        # 256x256x64  ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

x = Conv2D(filters=1, kernel_size=1)(x)                                         # 256x256x64  ->  256x256x1
OUTPUT = Activation(activation='sigmoid', name='output-layer')(x)               # 256x256x1   ->  256x256x1

CONFIG_NAME = 'tumorNet_v1.json'

### **tumorNet_v2**

In [ ]:
INPUT = Input(shape=[256, 256, 1], name='input-layer')

# down-sampling
x_1, down_1 = down_sampling_block(INPUT, 64)                                    # 256x256x1   ->  ( 256x256x64  | 128x128x64)
x_2, down_2 = down_sampling_block(down_1, 128)                                  # 128x128x64  ->  ( 128x128x128 | 64x64x128 )
x_3, down_3 = down_sampling_block(down_2, 256)                                  # 64x64x128   ->  ( 64x64x256   | 32x32x256 )
x_4, down_4 = down_sampling_block(down_3, 512)                                  # 32x32x256   ->  ( 32x32x512   | 16x16x512 )
x_5, down_5 = down_sampling_block(down_4, 1024)                                 # 16x16x512   ->  ( 16x16x1024  |  8x8x1024 )

# up-sampling
g_5, up_1 = up_sampling_block(down_5, 2048)                                     # 8x8x1024    ->  ( 8x8x2048    | 16x16x2048)
attention_1 = attention_gate(g_5, x_5)                                          # (     8x8x2048|  16x16x1024 ) ->  16x16x1024
concat = Concatenate(axis=-1)([attention_1, up_1])                              # (   16x16x1024|   16x16x2048) ->  16x16x3072

g_4, up_2 = up_sampling_block(down_4, 1024)                                     # 16x16x3072  ->  ( 16x16x1024  | 32x32x1024)
attention_2 = attention_gate(g_4, x_4)                                          # (   16x16x1024|   32x32x512 ) ->  32x32x512
concat = Concatenate(axis=-1)([attention_2, up_2])                              # (   32x32x512 |   32x32x1024) ->  32x32x1536

g_3, up_3 = up_sampling_block(concat, 512)                                      # 32x32x1536  ->  ( 32x32x512   | 64x64x512 )
attention_3 = attention_gate(g_3, x_3)                                          # (   32x32x512 |   64x64x256 ) ->  64x64x256
concat = Concatenate(axis=-1)([attention_3, up_3])                              # (   64x64x256 |   64x64x512 ) ->  64x64x768

g_2, up_4 = up_sampling_block(concat, 256)                                      # 64x64x768   ->  ( 64x64x256   | 128x128x256)
attention_4 = attention_gate(g_2, x_2)                                          # (   64x64x256 | 128x128x128 ) ->  128x128x128
concat = Concatenate(axis=-1)([attention_4, up_4])                              # ( 128x128x128 | 128x128x256 ) ->  128x128x384

g_1, up_5 = up_sampling_block(concat, 128)                                      # 128x128x384 ->  ( 128x128x128 | 256x256x128)
attention_5 = attention_gate(g_1, x_1)                                          # ( 128x128x128 | 256x256x64  ) ->  256x256x64
concat = Concatenate(axis=-1)([attention_5, up_5])                              # ( 256x256x64  | 256x256x128 ) ->  256x256x192

# final block
x = Conv2D(filters=64, kernel_size=3, padding='same')(concat)                   # 256x256x192 ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)                        # 256x256x64  ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

x = Conv2D(filters=1, kernel_size=1)(x)                                         # 256x256x64  ->  256x256x1
OUTPUT = Activation(activation='sigmoid', name='output-layer')(x)               # 256x256x1   ->  256x256x1

CONFIG_NAME = 'tumorNet_v2.json'

### **tumorNet_v3**

In [6]:
INPUT = Input(shape=[256, 256, 1], name='input-layer')

# down-sampling
x_1, down_1 = down_sampling_block(INPUT, 64)                                    # 256x256x1   ->  ( 256x256x64  | 128x128x64)
x_2, down_2 = down_sampling_block(down_1, 128)                                  # 128x128x64  ->  ( 128x128x128 | 64x64x128 )
x_3, down_3 = down_sampling_block(down_2, 256)                                  # 64x64x128   ->  ( 64x64x256   | 32x32x256 )
x_4, down_4 = down_sampling_block(down_3, 512)                                  # 32x32x256   ->  ( 32x32x512   | 16x16x512 )
x_5, down_5 = down_sampling_block(down_4, 1024)                                 # 16x16x512   ->  ( 16x16x1024  |  8x8x1024 )

# up-sampling
g_5, up_1 = up_sampling_block(down_5, 2048)                                     # 8x8x1024    ->  ( 8x8x2048    | 16x16x2048)
attention_1 = attention_gate(g_5, x_5)                                          # (     8x8x2048|  16x16x1024 ) ->  16x16x1024
concat = Concatenate(axis=-1)([attention_1, up_1])                              # (   16x16x1024|   16x16x2048) ->  16x16x3072

g_4, up_2 = up_sampling_block(down_4, 1024)                                     # 16x16x3072  ->  ( 16x16x1024  | 32x32x1024)
attention_2 = attention_gate(g_4, x_4)                                          # (   16x16x1024|   32x32x512 ) ->  32x32x512
concat = Concatenate(axis=-1)([attention_2, up_2])                              # (   32x32x512 |   32x32x1024) ->  32x32x1536

g_3, up_3 = up_sampling_block(concat, 512)                                      # 32x32x1536  ->  ( 32x32x512   | 64x64x512 )
attention_3 = attention_gate(g_3, x_3)                                          # (   32x32x512 |   64x64x256 ) ->  64x64x256
concat = Concatenate(axis=-1)([attention_3, up_3])                              # (   64x64x256 |   64x64x512 ) ->  64x64x768

g_2, up_4 = up_sampling_block(concat, 256)                                      # 64x64x768   ->  ( 64x64x256   | 128x128x256)
attention_4 = attention_gate(g_2, x_2)                                          # (   64x64x256 | 128x128x128 ) ->  128x128x128
concat = Concatenate(axis=-1)([attention_4, up_4])                              # ( 128x128x128 | 128x128x256 ) ->  128x128x384

g_1, up_5 = up_sampling_block(concat, 128)                                      # 128x128x384 ->  ( 128x128x128 | 256x256x128)
attention_5 = attention_gate(g_1, x_1)                                          # ( 128x128x128 | 256x256x64  ) ->  256x256x64
concat = Concatenate(axis=-1)([attention_5, up_5])                              # ( 256x256x64  | 256x256x128 ) ->  256x256x192

# final block
x = Conv2D(filters=64, kernel_size=3, padding='same')(concat)                   # 256x256x192 ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

x = Conv2D(filters=64, kernel_size=3, padding='same')(x)                        # 256x256x64  ->  256x256x64
x = BatchNormalization(axis=-1)(x)                                              # 256x256x64  ->  256x256x64
x = Activation(activation='relu')(x)                                            # 256x256x64  ->  256x256x64

OUTPUT = Conv2D(filters=1, kernel_size=1, name='output-layer')(x)               # 256x256x64  ->  256x256x1

CONFIG_NAME = 'tumorNet_v3.json'

### **Generate Config File**

##### **Validation**

In [7]:
tf.keras.backend.clear_session()
MODEL = tf.keras.Model(inputs=INPUT, outputs=OUTPUT, name='TumorDetector2.0')

# validating inference
BUILD = MODEL(tf.ones(shape=[1, 256, 256, 1]))
BUILD.shape

TensorShape([1, 256, 256, 1])

##### **Config File**

In [8]:
# generating a json file of
# model architecture
json = MODEL.to_json()

with open(os.path.join(OUTPUT_PATH, CONFIG_NAME), 'w') as json_file:
  json_file.write(json)

with open(os.path.join(OUTPUT_PATH, CONFIG_NAME), 'r') as config:
  MODEL_TESTER = tf.keras.models.model_from_json(config.read())

# model summary
MODEL_TESTER.summary()

c:\Users\Matheus\anaconda3\envs\tumor2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Model: "TumorDetector2.0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input-layer (InputLayer)        [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input-layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           batch_normalization[0][0]        
___________________________________________________________________________________